In [21]:
import pandas as pd
import pickle

df = pd.read_csv("C:/Users/wasadmin/Desktop/Starwarsapi3/GuidedAssignment3/troop_movements.csv", index_col=False)

df2 = df.groupby(['empire_or_resistance']).size()

info = [['empire',df2[0]], ['resistance',df2[1]]]
df3 = pd.DataFrame(info, columns = ['empire_or_resistance', 'count'])
df3


,empire_or_resistance,count
0,empire,491
1,resistance,509


In [23]:
df2 = df.groupby(['homeworld'])['homeworld'].count()
df2 = df2.to_frame()
df2 = df2.rename(columns={"homeworld": "Homeworld", "homeworld":"count"})
df2 = df2.reset_index()
df2

,homeworld,count
0,Alderaan,23
1,Aleen Minor,21
2,Bestine IV,19
3,Cerea,18
4,Champala,29
5,Chandrila,29
6,Concord Dawn,26
7,Corellia,27
8,Dagobah,19
9,Dathomir,28


In [24]:
df2 = df.groupby(['unit_type'])['unit_type'].count()
df2 = df2.to_frame()
df2 = df2.rename(columns={"unit_type": "Unit_type", "unit_type":"count"})
df2 = df2.reset_index()
df2

,unit_type,count
0,at-at,125
1,at-st,107
2,resistance_soldier,116
3,stormtrooper,124
4,tie_fighter,135
5,tie_silencer,120
6,unknown,154
7,x-wing,119
